In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
import gc

from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder
from sklearn.metrics import mean_squared_error

import lightgbm as lgb

import joblib

# 맥에서 글씨 깨짐 방지
from matplotlib import rc
rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False

/opt/anaconda3/lib/python3.11/site-packages/dask/dataframe/_pyarrow_compat.py:17: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 12.0.1. Please consider upgrading.
  warnings.warn(


# data preprocess

In [3]:
df = pd.read_csv('../data/preprocessed/34_apart_price_mean.csv')

/var/folders/zt/1l6rrzf977nbpmcz1d2j0vd40000gn/T/ipykernel_48363/3919769212.py:1: DtypeWarning: Columns (16,17,36) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/preprocessed/34_apart_price_mean.csv')


In [4]:
df.shape

(1128094, 105)

In [5]:
# 일부 feature 제거 - 기준은 내마음대로

df.drop(['해제사유발생일', '등기신청일자', '거래유형', '중개사소재지', 'k-단지분류(아파트,주상복합등등)', 'k-전화번호',
         'k-팩스번호', '단지소개기존clob', 'k-세대타입(분양형태)', 'k-복도유형', 'k-난방방식', 'k-전체동수', 'k-전체세대수',
         'k-사용검사일-사용승인일', 'k-관리비부과면적', 'k-전용면적별세대현황(60이하)', 'k-전용면적별세대현황(60~85이하)',
         'k-85~135이하', 'k-135초과', 'k-홈페이지', 'k-등록일자', 'k-수정일자', '고용보험관리번호', '경비비관리형태',
         '기타/의무/임대/임의=1/2/3/4', '단지승인일', '사용허가여부', '관리비 업로드', '단지신청일'], axis=1, inplace=True)

In [7]:
df.columns

Index(['시군구', '번지', '본번', '부번', '아파트명', '전용면적', '계약년월', '계약일', '층', '건축년도',
       '도로명', 'k-관리방식', 'k-건설사(시공사)', 'k-시행사', 'k-연면적', 'k-주거전용면적', '세대전기계약방법',
       '청소비관리형태', '건축면적', '주차대수', '좌표X', '좌표Y', 'target', 'is_test', '시군구 번지',
       '좌표X,좌표Y', '가장 가까운 거리', 'index', '역사_ID', '역사명', '호선', '위도', '경도',
       '인근 지하철 역 개수', '가장 가까운 버스 정류장 거리', '가장 가까운 버스 정류장 index',
       '가장 가까운 버스 정류장 노드 ID', '가장 가까운 버스 정류소번호', '가장 가까운 버스 정류소명',
       '가장 가까운 버스 정류소 타입', '가장 가까운 버스 정류장 X좌표', '가장 가까운 버스 정류장 Y좌표',
       '인근 버스 정류장 개수', '계약년', '계약월', 'GDP', '한국은행 기준금리', '기대 인플레이션', '지가지수',
       '아파트 인허가', '미분양', '거래량', '건설사 랭킹', '구', '동', '구별 지가지수', '공시지가 평균',
       '매수우위지수', '건물나이', '30년이상50년이하', '구매력지수', '거래활발지수', '매매가격 지수 증감률',
       '매매 대비 전세가격 비율', '아파트 카테고리', '지하철 카테고리', '구 카테고리', '건설사 카테고리',
       '가장 가까운 다리와의 거리', '가장 가까운 다리 index', '가장 가까운 다리', '인근 한강다리 개수',
       '인근 다리 개수 1개이상 3개 이하', '학군', '매매가격 지수', '아파트 평균 가격'],
      dtype='object')

In [8]:
# 추가로 제거가 필요한 feature 제거
df.drop(['시군구', '번지', '본번', '부번', '아파트명', '도로명', 'k-관리방식', 'k-건설사(시공사)', 'k-시행사', 'k-연면적', 'k-주거전용면적',
         '세대전기계약방법', '청소비관리형태', '건축면적', '주차대수', '시군구 번지', '좌표X,좌표Y', 'index', '역사_ID', '역사명', '위도', '경도',
         '가장 가까운 버스 정류장 index', '가장 가까운 버스 정류장 노드 ID', '가장 가까운 버스 정류소번호', '가장 가까운 버스 정류소명',
         '가장 가까운 버스 정류소 타입', '가장 가까운 버스 정류장 X좌표', '가장 가까운 버스 정류장 Y좌표', '동', 
         '구', '호선'], axis=1, inplace=True)

In [9]:
df.columns

Index(['전용면적', '계약년월', '계약일', '층', '건축년도', '좌표X', '좌표Y', 'target', 'is_test',
       '가장 가까운 거리', '인근 지하철 역 개수', '가장 가까운 버스 정류장 거리', '인근 버스 정류장 개수', '계약년',
       '계약월', 'GDP', '한국은행 기준금리', '기대 인플레이션', '지가지수', '아파트 인허가', '미분양', '거래량',
       '건설사 랭킹', '구별 지가지수', '공시지가 평균', '매수우위지수', '건물나이', '30년이상50년이하', '구매력지수',
       '거래활발지수', '매매가격 지수 증감률', '매매 대비 전세가격 비율', '아파트 카테고리', '지하철 카테고리',
       '구 카테고리', '건설사 카테고리', '가장 가까운 다리와의 거리', '가장 가까운 다리 index', '가장 가까운 다리',
       '인근 한강다리 개수', '인근 다리 개수 1개이상 3개 이하', '학군', '매매가격 지수', '아파트 평균 가격'],
      dtype='object')

In [10]:
df.shape

(1128094, 44)

In [13]:
# 계약월 변수 생성
df['계약월'] = df['계약년월'].astype(str).str[4:].astype(int)

# 계약년월, 계약일, 건축년도, 계약년 변수 제거
df.drop(['계약년월', '계약일'], axis=1, inplace=True)

df['계약월_sin'] = np.sin(2 * np.pi * df['계약월'] / 12)
df.drop(['계약월'], axis=1, inplace=True)
df

,전용면적,층,건축년도,좌표X,좌표Y,target,is_test,가장 가까운 거리,인근 지하철 역 개수,가장 가까운 버스 정류장 거리,...,건설사 카테고리,가장 가까운 다리와의 거리,가장 가까운 다리 index,가장 가까운 다리,인근 한강다리 개수,인근 다리 개수 1개이상 3개 이하,학군,매매가격 지수,아파트 평균 가격,계약월_sin
0,79.97,3,1987,127.05721,37.476763,124000.0,0,1127.738351,2.0,61.89584,...,기타,5581.622466,17.0,청담대교,0.0,0,8.0,67.392545,104403.465347,-2.449294e-16
1,79.97,4,1987,127.05721,37.476763,123500.0,0,1127.738351,2.0,61.89584,...,기타,5581.622466,17.0,청담대교,0.0,0,8.0,67.392545,104403.465347,-2.449294e-16
2,54.98,5,1987,127.05721,37.476763,91500.0,0,1127.738351,2.0,61.89584,...,기타,5581.622466,17.0,청담대교,0.0,0,8.0,67.392545,104403.465347,-2.449294e-16
3,79.97,4,1987,127.05721,37.476763,130000.0,0,1127.738351,2.0,61.89584,...,기타,5581.622466,17.0,청담대교,0.0,0,8.0,69.049924,104403.465347,5.000000e-01
4,79.97,2,1987,127.05721,37.476763,117000.0,0,1127.738351,2.0,61.89584,...,기타,5581.622466,17.0,청담대교,0.0,0,8.0,69.049924,104403.465347,5.000000e-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1128089,84.65,13,2014,127.10672,37.618870,NaN,1,732.482898,4.0,59.47567,...,기타,5918.624352,23.0,구리암사대교,0.0,0,1.0,91.623694,40355.229672,-5.000000e-01
1128090,84.62,12,2014,127.10672,37.618870,NaN,1,732.482898,4.0,59.47567,...,기타,5918.624352,23.0,구리암사대교,0.0,0,1.0,91.623694,40355.229672,-5.000000e-01
1128091,101.65,12,2014,127.10672,37.618870,NaN,1,732.482898,4.0,59.47567,...,기타,5918.624352,23.0,구리암사대교,0.0,0,1.0,91.066919,40355.229672,-8.660254e-01
1128092,84.94,18,2014,127.10672,37.618870,NaN,1,732.482898,4.0,59.47567,...,기타,5918.624352,23.0,구리암사대교,0.0,0,1.0,90.861265,40355.229672,-1.000000e+00


In [14]:
df.columns

Index(['전용면적', '층', '건축년도', '좌표X', '좌표Y', 'target', 'is_test', '가장 가까운 거리',
       '인근 지하철 역 개수', '가장 가까운 버스 정류장 거리', '인근 버스 정류장 개수', '계약년', 'GDP',
       '한국은행 기준금리', '기대 인플레이션', '지가지수', '아파트 인허가', '미분양', '거래량', '건설사 랭킹',
       '구별 지가지수', '공시지가 평균', '매수우위지수', '건물나이', '30년이상50년이하', '구매력지수', '거래활발지수',
       '매매가격 지수 증감률', '매매 대비 전세가격 비율', '아파트 카테고리', '지하철 카테고리', '구 카테고리',
       '건설사 카테고리', '가장 가까운 다리와의 거리', '가장 가까운 다리 index', '가장 가까운 다리',
       '인근 한강다리 개수', '인근 다리 개수 1개이상 3개 이하', '학군', '매매가격 지수', '아파트 평균 가격',
       '계약월_sin'],
      dtype='object')

# scaling

In [15]:
def scailing(col, scaler_type):
    if scaler_type == 'min_max': scaler = MinMaxScaler()
    else : scaler = StandardScaler()

    return scaler.fit_transform(df[[col]])

cols = ['전용면적', '층', '건축년도', '좌표X', '좌표Y', '가장 가까운 거리',
       '인근 지하철 역 개수', '가장 가까운 버스 정류장 거리', '인근 버스 정류장 개수', 'GDP',
       '한국은행 기준금리', '기대 인플레이션', '지가지수', '아파트 인허가', '미분양', '거래량',
       '구별 지가지수', '공시지가 평균', '매수우위지수', '건물나이', '구매력지수', '거래활발지수',
       '매매가격 지수 증감률', '매매 대비 전세가격 비율', '가장 가까운 다리와의 거리', '매매가격 지수',
       '아파트 평균 가격', '계약월_sin']

for col in cols:
    print(col)
    scaler_type = 'min_max'
    if col == '계약월_sin': scaler_type = 'standard'

    df[[col]] = scailing(col, scaler_type)

전용면적
층
건축년도
좌표X
좌표Y
가장 가까운 거리
인근 지하철 역 개수
가장 가까운 버스 정류장 거리
인근 버스 정류장 개수
GDP
한국은행 기준금리
기대 인플레이션
지가지수
아파트 인허가
미분양
거래량
구별 지가지수
공시지가 평균
매수우위지수
건물나이
구매력지수
거래활발지수
매매가격 지수 증감률
매매 대비 전세가격 비율
가장 가까운 다리와의 거리
매매가격 지수
아파트 평균 가격
계약월_sin


In [16]:
df.columns

Index(['전용면적', '층', '건축년도', '좌표X', '좌표Y', 'target', 'is_test', '가장 가까운 거리',
       '인근 지하철 역 개수', '가장 가까운 버스 정류장 거리', '인근 버스 정류장 개수', '계약년', 'GDP',
       '한국은행 기준금리', '기대 인플레이션', '지가지수', '아파트 인허가', '미분양', '거래량', '건설사 랭킹',
       '구별 지가지수', '공시지가 평균', '매수우위지수', '건물나이', '30년이상50년이하', '구매력지수', '거래활발지수',
       '매매가격 지수 증감률', '매매 대비 전세가격 비율', '아파트 카테고리', '지하철 카테고리', '구 카테고리',
       '건설사 카테고리', '가장 가까운 다리와의 거리', '가장 가까운 다리 index', '가장 가까운 다리',
       '인근 한강다리 개수', '인근 다리 개수 1개이상 3개 이하', '학군', '매매가격 지수', '아파트 평균 가격',
       '계약월_sin'],
      dtype='object')

# category

In [17]:
#df['30년이상50년이하'] = df['30년이상50년이하'].astype('category')
df['아파트 카테고리'] = df['아파트 카테고리'].astype('category')
df['지하철 카테고리'] = df['지하철 카테고리'].astype('category')
df['구 카테고리'] = df['구 카테고리'].astype('category')
df['건설사 카테고리'] = df['건설사 카테고리'].astype('category')
df['인근 다리 개수 1개이상 3개 이하'] = df['인근 다리 개수 1개이상 3개 이하'].astype('category')
df['학군'] = df['학군'].astype('category')

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1128094 entries, 0 to 1128093
Data columns (total 42 columns):
 #   Column               Non-Null Count    Dtype   
---  ------               --------------    -----   
 0   전용면적                 1128094 non-null  float64 
 1   층                    1128094 non-null  float64 
 2   건축년도                 1128094 non-null  float64 
 3   좌표X                  1128094 non-null  float64 
 4   좌표Y                  1128094 non-null  float64 
 5   target               1118822 non-null  float64 
 6   is_test              1128094 non-null  int64   
 7   가장 가까운 거리            1128094 non-null  float64 
 8   인근 지하철 역 개수          1128094 non-null  float64 
 9   가장 가까운 버스 정류장 거리     1128094 non-null  float64 
 10  인근 버스 정류장 개수         1128094 non-null  float64 
 11  계약년                  1128094 non-null  int64   
 12  GDP                  1128094 non-null  float64 
 13  한국은행 기준금리            1128094 non-null  float64 
 14  기대 인플레이션             1128094 non-n

# train, test dataset

## feature drop

In [19]:
train_df = df[df['is_test']==0]
test_df = df[df['is_test']==1]

In [20]:
train_df.shape

(1118822, 42)

In [21]:
train_df.columns

Index(['전용면적', '층', '건축년도', '좌표X', '좌표Y', 'target', 'is_test', '가장 가까운 거리',
       '인근 지하철 역 개수', '가장 가까운 버스 정류장 거리', '인근 버스 정류장 개수', '계약년', 'GDP',
       '한국은행 기준금리', '기대 인플레이션', '지가지수', '아파트 인허가', '미분양', '거래량', '건설사 랭킹',
       '구별 지가지수', '공시지가 평균', '매수우위지수', '건물나이', '30년이상50년이하', '구매력지수', '거래활발지수',
       '매매가격 지수 증감률', '매매 대비 전세가격 비율', '아파트 카테고리', '지하철 카테고리', '구 카테고리',
       '건설사 카테고리', '가장 가까운 다리와의 거리', '가장 가까운 다리 index', '가장 가까운 다리',
       '인근 한강다리 개수', '인근 다리 개수 1개이상 3개 이하', '학군', '매매가격 지수', '아파트 평균 가격',
       '계약월_sin'],
      dtype='object')

In [22]:
train_df_dropped = train_df.drop(['is_test', '지가지수', '아파트 인허가', '미분양', '건설사 랭킹',
                                  '건물나이', '구매력지수', '거래활발지수', '매매가격 지수 증감률', 
                                  '매매 대비 전세가격 비율', '구 카테고리', '건설사 카테고리', '가장 가까운 다리와의 거리',
                                  '가장 가까운 다리 index', '가장 가까운 다리', '인근 한강다리 개수', 
                                  '인근 다리 개수 1개이상 3개 이하', '학군', '매매가격 지수'], axis=1)

test_X = test_df.drop(['is_test', 'target', '지가지수', '아파트 인허가', '미분양', '건설사 랭킹',
                                '건물나이', '구매력지수', '거래활발지수', '매매가격 지수 증감률', 
                                '매매 대비 전세가격 비율', '구 카테고리', '건설사 카테고리', '가장 가까운 다리와의 거리',
                                '가장 가까운 다리 index', '가장 가까운 다리', '인근 한강다리 개수', 
                                '인근 다리 개수 1개이상 3개 이하', '학군', '매매가격 지수'], axis=1)

X = train_df_dropped.drop(['target'], axis=1)
y = train_df_dropped[['target']]

train_X, valid_X, train_y, valid_y = train_test_split(X, y, train_size=0.95, test_size=0.05, random_state=74)

train_X.columns

Index(['전용면적', '층', '건축년도', '좌표X', '좌표Y', '가장 가까운 거리', '인근 지하철 역 개수',
       '가장 가까운 버스 정류장 거리', '인근 버스 정류장 개수', '계약년', 'GDP', '한국은행 기준금리',
       '기대 인플레이션', '거래량', '구별 지가지수', '공시지가 평균', '매수우위지수', '30년이상50년이하',
       '아파트 카테고리', '지하철 카테고리', '아파트 평균 가격', '계약월_sin'],
      dtype='object')

In [23]:
params = {
    'n_estimators': 100000,
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'rmse',
    'metric_freq': 20,
    'device': 'gpu',
    'verbosity': 0
}

model = lgb.LGBMRegressor(n_estimators=100000,
                          metric='rmse', data_sample_strategy='goss'
                          )

model.fit(
    train_X, train_y,
    eval_set = [(train_X, train_y), (valid_X, valid_y)],
    eval_metric='rmse',
    callbacks=[lgb.early_stopping(stopping_rounds=60),
               lgb.log_evaluation(period=10, show_stdv=True)]
)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.014254 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2705
[LightGBM] [Info] Number of data points in the train set: 1062880, number of used features: 22
[LightGBM] [Info] Using GOSS
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Start training from score 57982.908459
Training until validation scores don't improve for 60 rounds
[10]	training's rmse: 23611.4	valid_1's rmse: 24208.7
[20]	training's rmse: 16295.5	valid_1's rmse: 16769.1
[30]	training's rmse: 13750.6	valid_1's rmse: 14164
[40]	training's rmse: 12570.7	valid_1's rmse: 12986.6
[50]	training's rmse: 11847.2	valid_1's rmse: 12237.2
[60]	training's rmse: 11306.6	valid_1's rmse: 11687
[70]	training's rm

LGBMRegressor(data_sample_strategy='goss', metric='rmse', n_estimators=100000)

In [24]:
train_X.columns

Index(['전용면적', '층', '건축년도', '좌표X', '좌표Y', '가장 가까운 거리', '인근 지하철 역 개수',
       '가장 가까운 버스 정류장 거리', '인근 버스 정류장 개수', '계약년', 'GDP', '한국은행 기준금리',
       '기대 인플레이션', '거래량', '구별 지가지수', '공시지가 평균', '매수우위지수', '30년이상50년이하',
       '아파트 카테고리', '지하철 카테고리', '아파트 평균 가격', '계약월_sin'],
      dtype='object')

In [25]:
test_X.columns

Index(['전용면적', '층', '건축년도', '좌표X', '좌표Y', '가장 가까운 거리', '인근 지하철 역 개수',
       '가장 가까운 버스 정류장 거리', '인근 버스 정류장 개수', '계약년', 'GDP', '한국은행 기준금리',
       '기대 인플레이션', '거래량', '구별 지가지수', '공시지가 평균', '매수우위지수', '30년이상50년이하',
       '아파트 카테고리', '지하철 카테고리', '아파트 평균 가격', '계약월_sin'],
      dtype='object')

In [26]:
preds = model.predict(test_X)

In [27]:
preds

array([190937.6364006 , 318049.41245551, 333292.57113663, ...,
        86394.2819661 ,  74432.79253627,  74926.20154382])

In [28]:
preds = np.round(preds).astype(int)

In [29]:
preds_df = pd.DataFrame(preds, columns=['target'])

In [30]:
preds_df

,target
0,190938
1,318049
2,333293
3,299273
4,228520
...,...
9267,73878
9268,73342
9269,86394
9270,74433


In [52]:
joblib.dump(model, './v_34_2.pkl')

['./v_34_2.pkl']

In [53]:
preds_df.to_csv('../preds/v_34_2_preds.csv', index=False)